In [1]:
import numpy as np
from scipy.integrate import odeint

# Définir les paramètres du modèle
L = 1.0   # Longueur de la corde en mètres
T = 1.0   # Tension de la corde en newtons
rho = 1.0 # Densité linéique de la corde en kg/m
n = 10    # Nombre de modes de vibration à considérer
m = 100   # Nombre de points d'échantillonnage sur la corde

# Définir la fonction qui calcule les coefficients de la matrice modale
def compute_modal_coeffs(n, L):
    coeffs = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            coeffs[i, j] = np.sin((i+1) * np.pi * (j+1) / (n+1))
    coeffs /= np.sqrt(L)
    return coeffs

C = compute_modal_coeffs(n, L)

# Définir la fonction qui calcule la force de tension
def tension(x):
    return T

# Définir la fonction qui calcule la force de frottement
def friction(x, v):
    return 0.0

# Définir la fonction qui calcule les dérivées des états (vitesses et accélérations)
def derivatives(y, t):
    # Extraire les vecteurs d'états (vitesses et positions)
    u = y[:n]
    q = y[n:]

    # Calculer les coefficients de la matrice modale pour les positions de la corde
    Q = np.dot(C, q)

    # Calculer les forces de tension et de frottement
    f_tension = np.dot(C.T, np.vectorize(tension)(q))
    f_friction = np.dot(C.T, np.vectorize(friction)(q, u))

    # Calculer les accélérations
    a = np.dot(C.T, f_tension - f_friction) / rho

    # Assembler les dérivées des vecteurs d'états
    du_dt = -a
    dq_dt = u

    # Retourner les dérivées
    return np.concatenate((du_dt, dq_dt))



# Définir les conditions initiales
u0 = np.zeros(n)
q0 = np.sin(np.linspace(0, np.pi, m))
y0 = np.concatenate((u0, q0))

# Définir le vecteur de temps
t = np.linspace(0, 1, 1000)

# Résoudre les équations différentielles avec odeint
sol = odeint(derivatives, y0, t)


# Extraire les vecteurs d'états (vitesses et positions) de la solution
u = sol[:, :n]
q = sol[:, n:]

# Calculer les positions de la corde à différents instants
x = np.linspace(0, L, m)
positions = np.dot(C, q.T)

# Afficher les positions de la corde à différents instants
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ValueError: shapes (10,10) and (100,) not aligned: 10 (dim 1) != 100 (dim 0)